In [ ]:
!pip install keras
!pip install numpy
!pip install pandas
!pip install tensorflow
!pip install lime
!pip install json

In [ ]:
!pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import shutil
import random
import numpy as np
import json
import zipfile
import os
import pandas as pd
import PIL
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
import os
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense, Flatten
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import lime
from lime import lime_image
from PIL import Image

In [ ]:
!wget "http://seppe.net/aa/assignment2/images.zip"
!7z x images.zip

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
! rm images.zip

In [ ]:
import os
images_path = '/content/images'
dataset_path = '/content/gdrive/My Drive/dataset.json'

Extract the price categories and store them in a library where they are linked to images. (We will later need this as a dataframe)

In [ ]:
with open(dataset_path, "r") as f:
    data = json.load(f)
price_categories = {}
image_ids = []
for item in data:
    # Check if 'price_category' key is present in the 'item' dictionary
    if 'price_category' in item and item['price_category'] is not None:
        # Extract the price category
        price_category = item['price_category']['label']
        # Extract the image IDs from the "full_images" list and map them to their price category
        for img in item["more_details"]["full_images"]:
            image_id = img["image_id"]
            image_ids.append(image_id)
            price_categories[image_id] = price_category


Now I have a list of image id's and a library of price categories.
The file names of the images are just the id with .jpg behind it.
I want to create the training, validation and test set.
For this I need to make a pandas data frame with two columns, with the image file names and one with the categories.

First, we shuffle the image id's and then split them up in training, validation and test set lists of image id's.(The training, validation, test sizes are 70%, 20% and 10%. We could, however **change these values 0.7, 0.2, 0.1 to lower values if we want to use less of the data**.)

In [ ]:
random.shuffle(image_ids)  # shuffle image id's

train_size = 0.7
val_size = 0.2
test_size = 0.1

train_image_ids = image_ids[:int(len(image_ids) * train_size)]
val_image_ids = image_ids[int(len(image_ids) * train_size):int(len(image_ids) * (train_size + val_size))]
test_image_ids = image_ids[int(len(image_ids) * (train_size + val_size)):int(len(image_ids) * (train_size + val_size + test_size))]

Now we create lists of the filenames of these images.

In [ ]:
train_filenames = [f"{image_id}.jpg" for image_id in train_image_ids]
val_filenames = [f"{image_id}.jpg" for image_id in val_image_ids]
test_filenames = [f"{image_id}.jpg" for image_id in test_image_ids]


Now, we create a pandas dataframe with two columns: "filename" and "price_category".

In [ ]:
train_df = pd.DataFrame(
    {"id": train_filenames, "label": [price_categories[image_id] for image_id in train_image_ids]})
val_df = pd.DataFrame(
    {"id": val_filenames, "label": [price_categories[image_id] for image_id in val_image_ids]})
test_df = pd.DataFrame(
    {"id": test_filenames, "label": [price_categories[image_id] for image_id in test_image_ids]})

We set the paths for the training, validation, and test data directories and create the directories.

We set the paths for the training, validation, and test data directories and create the directories.

In [ ]:
# delete the directories if they are there already
if os.path.exists('/content/images/train'):
    shutil.rmtree('/content/images/train')
else:
    print(f"The folder '{'/content/images/train'}' does not exist yet.")

if os.path.exists('/content/images/val'):
    shutil.rmtree('/content/images/val')
else:
    print(f"The folder '{'/content/images/val'}' does not exist yet.")

if os.path.exists('/content/images/test'):
    shutil.rmtree('/content/images/test')
else:
    print(f"The folder '{'/content/images/test'}' does not exist yet.")

# Set the paths for the training, validation, and test data directories
train_data_dir = os.path.join(images_path, "train")
val_data_dir = os.path.join(images_path, "val")
test_data_dir = os.path.join(images_path, "test")

# Create the directories if they don't exist already
!mkdir -p "{train_data_dir}"
!mkdir -p "{val_data_dir}"
!mkdir -p "{test_data_dir}"

We move the image files to the appropriate directories.

In [ ]:
from PIL import Image

def is_image_corrupted(image_path):
    try:
        Image.open(image_path)
        return False
    except (IOError, SyntaxError):
        return True


In [ ]:
for filename in train_filenames:
    src_path = os.path.join(images_path, filename)
    dst_path = os.path.join(train_data_dir, filename)
    is_corrupted = is_image_corrupted(src_path)
    if is_corrupted:
        print(f"corrupted image: {filename}")
    else:
        shutil.copy(src_path, dst_path)

for filename in val_filenames:
    src_path = os.path.join(images_path, filename)
    dst_path = os.path.join(val_data_dir, filename)
    is_corrupted = is_image_corrupted(src_path)
    if is_corrupted:
        print(f"corrupted image: {filename}")
    else:
        shutil.copy(src_path, dst_path)

for filename in test_filenames:
    src_path = os.path.join(images_path, filename)
    dst_path = os.path.join(test_data_dir, filename)
    is_corrupted = is_image_corrupted(src_path)
    if is_corrupted:
        print(f"corrupted image: {filename}")
    else:
        shutil.copy(src_path, dst_path)

corrupted image: 4215425.jpg


Now we want to preprocess the data and then generate it.


We first define the image dimensions and the batch size. **(Change this, see what works best)**

In [ ]:
img_width, img_height = 224, 224
batch_size = 256

We use ImageDataGenerator for preprocessing including rotation augmentation.

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255, rotation_range=20, fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

We generate the training, validation, and testing datasets.

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
        dataframe = train_df,
        directory='/content/images/train',
        target_size=(224,224),
        x_col="id",
        y_col="label",
        seed=42,
        shuffle=True,
        class_mode="categorical",
        batch_size=256)

validation_generator = val_datagen.flow_from_dataframe(
        dataframe = val_df,
        directory='/content/images/val',
        target_size=(224,224),
        x_col="id",
        y_col="label",
        seed=42,
        shuffle=True,
        class_mode="categorical",
        batch_size=256)

test_generator = test_datagen.flow_from_dataframe(
        dataframe = test_df,
        directory='/content/images/test',
        target_size=(224,224),
        x_col="id",
        y_col="label",
        seed=42,
        shuffle=False,
        class_mode="categorical",
        batch_size=256)

/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 1 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  warnings.warn(


Found 80961 validated image filenames belonging to 4 classes.
Found 23132 validated image filenames belonging to 4 classes.
Found 11566 validated image filenames belonging to 4 classes.


Now we can finally build our Neural Network. We created one by hand at first, and then used a pretrained model (VGG16) to compare the results.

Define the CNN structure.

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

Compile the model.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Train the model.(**Number of epochs can be changed**)

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=5,
                              validation_data=validation_generator, validation_steps=validation_generator.samples // batch_size)

<ipython-input-45-971392841fde>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=5,


Epoch 1/5
 98/316 [========>.....................] - ETA: 35:35 - loss: 1.3429 - accuracy: 0.4171

Evaluate the model.

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=test_generator.samples // batch_size)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)


<ipython-input-32-416c9d65b6c8>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_acc = model.evaluate_generator(test_generator, steps=test_generator.samples // batch_size)


Test accuracy: 0.4071180522441864
Test loss: 1.255115032196045


Interpreter LIME


In [ ]:
explainer = lime_image.LimeImageExplainer()


def predict_wrapper(images):
    # This function is a wrapper around the model's prediction function
    # It takes in a batch of images (N, 224, 224, 3) and returns the predicted probabilities (N, 5)
    return model.predict(images)


# Get an example image from the validation set
example_image_path = os.path.join(val_data_dir, val_filenames[0])
example_image = Image.open(example_image_path)

# Explain the model's prediction for the example image
explanation = explainer.explain_instance(np.array(example_image), predict_wrapper, top_labels=5)

# Show the LIME visualization for the explanation
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
img_boundry = mark_boundaries(temp/2 + 0.5, mask)
Image.fromarray((img_boundry*255).astype(np.uint8))

Now, we do the same thing but with the pretrained model VGG16.

We first preprocess and generate.

In [ ]:
# preprocessing according to VGG16
batch_size = 64
train_datagen = ImageDataGenerator(rescale=1. / 255, rotation_range=90,
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5,
                                     height_shift_range=0.5,
                                     horizontal_flip=True,
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(rescale=1. / 255,preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(rescale=1. / 255,preprocessing_function=preprocess_input)

# Generate the training, validation, and testing datasets (I followed the instructions on https://www.learndatasci.com/tutorials/hands-on-transfer-learning-keras/)
train_generator = train_datagen.flow_from_dataframe(
        dataframe = train_df,
        directory='/content/images/train',
        target_size=(224,224),
        x_col="id",
        y_col="label",
        seed=42,
        shuffle=True,
        class_mode="categorical",
        batch_size=64)

validation_generator = val_datagen.flow_from_dataframe(
        dataframe = val_df,
        directory='/content/images/val',
        target_size=(224,224),
        x_col="id",
        y_col="label",
        seed=42,
        shuffle=True,
        class_mode="categorical",
        batch_size=64)

test_generator = test_datagen.flow_from_dataframe(
        dataframe = test_df,
        directory='/content/images/test',
        target_size=(224,224),
        x_col="id",
        y_col="label",
        seed=42,
        shuffle=False,
        class_mode="categorical",
        batch_size=64)


Found 8096 validated image filenames belonging to 4 classes.
Found 2313 validated image filenames belonging to 4 classes.
Found 1157 validated image filenames belonging to 4 classes.


Load without the top layer input_shape(img_width, img_height, 3) (the 3 is for the colors rbg).

In [ ]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in vgg_model.layers:
    layer.trainable = False
    # This is to make sure we don't train these layers

58889256/58889256 [==============================] - 2s 0us/step


We add a new output layer with 4 categories. Then we create the new model with the pretrained layers and our new output layer.

In [ ]:
x = Flatten()(vgg_model.output)
output_layer = Dense(4, activation='softmax')(x)

model_VGG = Model(inputs=vgg_model.input, outputs=output_layer)

Compile the model.

In [ ]:
model_VGG.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Train the model. (**Number of epochs can be changed**)

In [ ]:
history = model_VGG.fit_generator(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=5,
                              validation_data=validation_generator, validation_steps=validation_generator.samples // batch_size)

<ipython-input-19-e56505b42464>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model_VGG.fit_generator(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=5,


Epoch 1/5
126/126 [==============================] - 409s 3s/step - loss: 1.4088 - accuracy: 0.3757 - val_loss: 1.5636 - val_accuracy: 0.4180
Epoch 2/5
126/126 [==============================] - 363s 3s/step - loss: 1.3792 - accuracy: 0.3729 - val_loss: 1.5009 - val_accuracy: 0.3637
Epoch 3/5
126/126 [==============================] - 384s 3s/step - loss: 1.3917 - accuracy: 0.3797 - val_loss: 1.9117 - val_accuracy: 0.3294
Epoch 4/5
126/126 [==============================] - 386s 3s/step - loss: 1.3997 - accuracy: 0.3744 - val_loss: 1.6004 - val_accuracy: 0.3832
Epoch 5/5
126/126 [==============================] - 371s 3s/step - loss: 1.3777 - accuracy: 0.3790 - val_loss: 1.8566 - val_accuracy: 0.3225


Evaluate the model.


In [ ]:
test_loss, test_acc = model_VGG.evaluate_generator(test_generator, steps=test_generator.samples // batch_size)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

<ipython-input-20-bee26fdafaac>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_acc = model_VGG.evaluate_generator(test_generator, steps=test_generator.samples // batch_size)


Test accuracy: 0.3220486044883728
Test loss: 1.8300116062164307


LIME interpreter

In [ ]:
explainer = lime_image.LimeImageExplainer()


def predict_wrapper_VGG(images):
    # This function is a wrapper around the model's prediction function
    # It takes in a batch of images (N, 224, 224, 3) and returns the predicted probabilities (N, 5)
    return model_VGG.predict(images)


# Get an example image from the validation set
example_image_path = os.path.join(val_data_dir, val_filenames[0])
example_image = Image.open(example_image_path)

# Explain the model's prediction for the example image
explanation = explainer.explain_instance(np.array(example_image), predict_wrapper_VGG, top_labels=5)

# Show the LIME visualization for the explanation
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
img_boundry = mark_boundaries(temp/2 + 0.5, mask)
Image.fromarray((img_boundry*255).astype(np.uint8))

Now, for the ResNet50 pretrained model

In [ ]:
from keras.applications import ResNet50

# Preprocessing
batch_size = 64
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=90,
    brightness_range=[0.1, 0.7],
    width_shift_range=0.5,
    height_shift_range=0.5,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.15,
    preprocessing_function=preprocess_input
)
val_datagen = ImageDataGenerator(rescale=1. / 255, preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(rescale=1. / 255, preprocessing_function=preprocess_input)

# Generate the training, validation, and testing datasets
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='/content/images/train',
    target_size=(224, 224),
    x_col="id",
    y_col="label",
    seed=42,
    shuffle=True,
    class_mode="categorical",
    batch_size=64
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='/content/images/val',
    target_size=(224, 224),
    x_col="id",
    y_col="label",
    seed=42,
    shuffle=True,
    class_mode="categorical",
    batch_size=64
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='/content/images/test',
    target_size=(224, 224),
    x_col="id",
    y_col="label",
    seed=42,
    shuffle=False,
    class_mode="categorical",
    batch_size=64
)

# Load the ResNet50 model
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the ResNet50 model
for layer in resnet_model.layers:
    layer.trainable = False

# Add your own classification layers on top of the ResNet50 model
x = Flatten()(resnet_model.output)
x = Dense(256, activation='relu')(x)
output_layer = Dense(4, activation='softmax')(x)

# Create the final model
model_resnet = Model(inputs=resnet_model.input, outputs=output_layer)

# Compile and train the model
model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_resnet = model_resnet.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Found 8096 validated image filenames belonging to 4 classes.
Found 2313 validated image filenames belonging to 4 classes.
Found 1157 validated image filenames belonging to 4 classes.
94765736/94765736 [==============================] - 3s 0us/step
Epoch 1/5
126/126 [==============================] - 383s 3s/step - loss: 2.1067 - accuracy: 0.3723 - val_loss: 1.4252 - val_accuracy: 0.4236
Epoch 2/5
126/126 [==============================] - 374s 3s/step - loss: 1.2843 - accuracy: 0.3897 - val_loss: 1.2930 - val_accuracy: 0.3238
Epoch 3/5
126/126 [==============================] - 372s 3s/step - loss: 1.2664 - accuracy: 0.3893 - val_loss: 1.3119 - val_accuracy: 0.4240
Epoch 4/5
126/126 [==============================] - 354s 3s/step - loss: 1.2572 - accuracy: 0.3979 - val_loss: 1.2930 - val_accuracy: 0.4236
Epoch 5/5
126/126 [==============================] - 354s 3s/step - loss: 1.2515 - accuracy: 0.4011 - val_loss: 1.2611 - val_accuracy: 0.4240


In [ ]:
test_loss, test_acc = model_resnet.evaluate_generator(test_generator, steps=test_generator.samples // batch_size)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

<ipython-input-23-3eab6bae4f73>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_acc = model_resnet.evaluate_generator(test_generator, steps=test_generator.samples // batch_size)


Test accuracy: 0.40625
Test loss: 1.2785124778747559


Now for EfficientNet

In [ ]:
!pip install efficientnet
from keras.applications import EfficientNetB0


# Preprocessing
batch_size = 64
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=90,
    brightness_range=[0.1, 0.7],
    width_shift_range=0.5,
    height_shift_range=0.5,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.15,
    preprocessing_function=preprocess_input
)
val_datagen = ImageDataGenerator(rescale=1. / 255, preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(rescale=1. / 255, preprocessing_function=preprocess_input)

# Generate the training, validation, and testing datasets
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='/content/images/train',
    target_size=(224, 224),
    x_col="id",
    y_col="label",
    seed=42,
    shuffle=True,
    class_mode="categorical",
    batch_size=64
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='/content/images/val',
    target_size=(224, 224),
    x_col="id",
    y_col="label",
    seed=42,
    shuffle=True,
    class_mode="categorical",
    batch_size=64
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='/content/images/test',
    target_size=(224, 224),
    x_col="id",
    y_col="label",
    seed=42,
    shuffle=False,
    class_mode="categorical",
    batch_size=64
)

# Load the EfficientNetB0 model
efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the EfficientNetB0 model
for layer in efficientnet_model.layers:
    layer.trainable = False

# Add your own classification layers on top of the EfficientNetB0 model
x = Flatten()(efficientnet_model.output)
x = Dense(256, activation='relu')(x)
output_layer = Dense(4, activation='softmax')(x)

# Create the final model
model_efficientnet = Model(inputs=efficientnet_model.input, outputs=output_layer)

# Compile and train the model
model_efficientnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_efficientnet = model_efficientnet.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.7 MB/s eta 0:00:00
Found 8096 validated image filenames belonging to 4 classes.
Found 2313 validated image filenames belonging to 4 classes.
Found 1157 validated image filenames belonging to 4 classes.
16705208/16705208 [==============================] - 1s 0us/step
Epoch 1/5
126/126 [==============================] - 378s 3s/step - loss: 3.7321 - accuracy: 0.3490 - val_loss: 1.2722 - val_accuracy: 0.4236
Epoch 2/5
126/126 [==============================] - 372s 3s/step - loss: 1.3177 - accuracy: 0.3643 - val_loss: 1.2846 - val_accuracy: 0.4232
Epoch 3/5
126/126 [==============================] - 350s 3s/step - loss: 1.3406 - accuracy: 0.3687 - val_loss: 1.2473 - val_accuracy: 0.4249
Epoch 4/5
126/126 [==============================] - 349s 3s/step - loss: 1.3323 - accuracy: 0.3594 - val_loss: 1.4003 - val_accuracy: 0.3190
Epoch

In [ ]:
test_loss, test_acc = model_efficientnet.evaluate_generator(test_generator, steps=test_generator.samples // batch_size)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

<ipython-input-25-a4aa1a68026b>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_acc = model_efficientnet.evaluate_generator(test_generator, steps=test_generator.samples // batch_size)


Test accuracy: 0.40625
Test loss: 1.2800960540771484
